In [62]:
import pandas as pd
from kblab import Archive
import requests
from requests.auth import HTTPBasicAuth
import time
from pathlib import Path
from bs4 import BeautifulSoup
romaner=[{"meta.host_title": "Welfare state analytics"},"https://datalab.kb.se"]

def kb_df(filter=romaner,max_number=5000):
    kb_data=pd.DataFrame()
    kb_data["Libris-ID"]=""
    kb_data["Datalab ID"]=""
    kb_data["has_content"]=""
    home_dir=str(Path.home())

    with open("/home/liamtabibzadeh/Documents/API_credentials.txt", 'r') as file:
        pw = file.read().replace('\n', '')
    a = Archive("https://datalab.kb.se", auth=("demo", pw))

    for package_id in a.search(filter[0], max=max_number):
        book_values=[]
        libris=None

        for x in a.get(package_id):
            if "mets.metadata" in x:
                    
                    for i in range(5):
                        backoff_time = 0.1 * (2 ** i)
                        meta=requests.get(f"{filter[1]}/{package_id}/{x}", auth=HTTPBasicAuth("demo", pw),stream=True)
                        if meta.status_code == 200:
                            libris=BeautifulSoup(meta.text, 'xml').find('mods:identifier', {'type': 'uri'}).text[21:]
                            break
                        else:
                            time.sleep(backoff_time)
                            if i==5:
                                print(f"Libris attempt failed for {package_id} even if metadata exists")
                    break
        book_values.append(libris)
        book_values.append(package_id)

        has_content=False
        for x in a.get(package_id):
            if "xml" in x:
                has_content=True
                break
        book_values.append(has_content)

        kb_data.loc[len(kb_data)]=book_values
    return kb_data



In [63]:
kb=kb_df()


In [27]:
meta=pd.read_excel("/home/liamtabibzadeh/Documents/kb info/meta.xlsx")

In [70]:
all_meta=pd.merge(kb,meta,how="left",on="Libris-ID")
metadata_missing_content=all_meta[all_meta["has_content"]==False] 

all_meta['has_content'] = all_meta['has_content'].map({False: "no",True:"yes"})
metadata_missing_content.to_excel("saknas.xlsx",index=False)
all_meta.to_excel("meta_data.xlsx",index=False)
